In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import csv
import unidecode
import math
import utm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def calculate_weighted_mean(data):
    data['FE_VIA'] = data['FE_VIA'].apply(lambda x: 1 if math.isnan(x) else x)
    data['FE_VIA'] = data['FE_VIA'].apply(lambda x: 1 if int(x) == 0 else x)
    data['MP'] = data['FE_VIA'] * data['DURACAO']
    return data

folder_data = "../data/"
arq17 = "dados17.csv"

data17 = pd.read_csv(folder_data + arq17, dtype={'ZONA_O': str, 'ZONA_D': str}, header=0,delimiter=";", low_memory=False) 
data17 = data17.dropna(subset=['CO_O_X'])

data17 = data17.drop(['ID_DOM', 'FE_DOM', 'VIA_BICI','TP_ESTBICI','F_FAM','FE_FAM','FAMILIA','NO_MORAF',
                      'CONDMORA','QT_BANHO','QT_EMPRE','QT_AUTO','QT_MICRO','QT_LAVALOU','QT_GEL1'], axis=1)

csv_file = folder_data + "regioes17.csv"
mydict = []
with open(csv_file, mode='r') as infile:
    reader = csv.reader(infile, delimiter=";")
    mydict = {rows[0]:rows[1] for rows in reader}

csv_file = folder_data + "zonas17.csv"
zonas_nomes = []
with open(csv_file, mode='r') as infile:
    reader = csv.reader(infile, delimiter=";")
    zonas_nomes = {rows[0]:rows[1] for rows in reader}
    
data17 = data17[data17['MUNI_O'] == 36]
data17 = data17[data17['MUNI_D'] == 36]

data17['NOME_O'] = data17['ZONA_O'].apply(lambda x: '' if pd.isnull(x) else mydict[x])
data17['NOME_D'] = data17['ZONA_D'].apply(lambda x: '' if pd.isnull(x) else mydict[x])

data17['ZONA_O'] = data17['ZONA_O'].apply(lambda x: '' if pd.isnull(x) else zonas_nomes[x])
data17['ZONA_D'] = data17['ZONA_D'].apply(lambda x: '' if pd.isnull(x) else zonas_nomes[x])

data17['NUM_TRANS'] = data17[['MODO1', 'MODO2','MODO3','MODO4']].count(axis=1)

data17 = calculate_weighted_mean(data17)

data17.head()

,ZONA,MUNI_DOM,CO_DOM_X,CO_DOM_Y,F_DOM,DOM,CD_ENTRE,DATA,TIPO_DOM,AGUA,...,TIPVG,PAG_VIAG,TP_ESAUTO,VL_EST,PE_BICI,ID_ORDEM,NOME_O,NOME_D,NUM_TRANS,MP
0,1,36,333743,7394463,1,1,1,6092017,1,1,...,3.0,NaN,NaN,NaN,1.0,1,Se,Se,1,221.32647
1,1,36,333743,7394463,0,1,1,6092017,1,1,...,3.0,NaN,NaN,NaN,1.0,2,Se,Se,1,221.32647
2,1,36,333743,7394463,0,1,1,6092017,1,1,...,1.0,2.0,NaN,NaN,NaN,3,Se,Pinheiros,1,944.12435
3,1,36,333743,7394463,0,1,1,6092017,1,1,...,1.0,2.0,NaN,NaN,NaN,4,Pinheiros,Perdizes,2,1132.94922
4,1,36,333743,7394463,0,1,1,6092017,1,1,...,2.0,NaN,NaN,NaN,NaN,5,Perdizes,Se,1,755.29948


In [3]:
modos17 = {0:'outros',1:'metro',2:'trem',3:'metro',4:'onibus',5:'onibus',6:'onibus',7:'fretado', 8:'escolar',9:'carro-dirigindo', 10: 'carro-passageiro', 11:'taxi', 12:'taxi-nao-convencional', 13:'moto', 14:'moto-passageiro', 15:'bicicleta', 16:'pe', 17: 'outros'}
data17['MODOPRIN'] = data17['MODOPRIN'].replace(modos17)

data17['MODOPRIN'].head()

0                       pe
1                       pe
2                    metro
3                    metro
4    taxi-nao-convencional
Name: MODOPRIN, dtype: object

In [4]:
modo_distri = data17[['MODOPRIN', 'NOME_O', 'FE_VIA']].groupby(['MODOPRIN','NOME_O']).sum().sort_values(by=['FE_VIA']).reset_index()
modo_distri.columns = ['MODOPRIN', 'NOME_O', 'FE_VIA_BIKE']
modo_distri = modo_distri[modo_distri['MODOPRIN'] == 'bicicleta']

modo_distri = modo_distri.set_index('NOME_O')


modo_distri

distrito = data17[['NOME_O', 'FE_VIA']].groupby(['NOME_O']).sum().sort_values(by=['FE_VIA']).reset_index()
distrito.columns = ['NOME_O', 'FE_VIA_TOTAL']
distrito = distrito.set_index('NOME_O')


modo_distri = modo_distri.join(distrito)

modo_distri = modo_distri.drop(['MODOPRIN'], axis=1)
modo_distri['PORCETAGEM'] = modo_distri['FE_VIA_BIKE'] / modo_distri['FE_VIA_TOTAL'] * 100
modo_distri.columns = ['Bicicleta', 'Total', 'Bicicleta (%)']
modo_distri = modo_distri.round(2)
modo_distri['Bicicleta'] = modo_distri['Bicicleta'].round(0)
modo_distri['Total'] = modo_distri['Total'].round(0)
modo_distri.sort_values(by=['Bicicleta (%)']).tail(10)

,Bicicleta,Total,Bicicleta (%)
NOME_O,,,
Itaim Bibi,11245.0,547034.0,2.06
Pinheiros,8215.0,366260.0,2.24
Pari,1711.0,72105.0,2.37
Socorro,3062.0,118084.0,2.59
Iguatemi,6567.0,211175.0,3.11
Vila Maria,7304.0,227562.0,3.21
Ipiranga,10386.0,318122.0,3.26
Jaguare,3293.0,98784.0,3.33
Jardim Helena,8629.0,204378.0,4.22


In [7]:

distrito_renda = data17[['NOME_O', 'RENDA_FA']].groupby(['NOME_O']).mean().sort_values(by=['RENDA_FA']).reset_index()
distrito_renda.columns = ['NOME_O', 'RENDA']
distrito_renda = distrito_renda.set_index('NOME_O')
distrito_renda


,RENDA
NOME_O,
Sao Rafael,2614.920579
Lajeado,2671.110391
Itaim Paulista,2769.636889
Cidade Tiradentes,2812.517945
Jardim Angela,2847.469419
Jaragua,2876.055420
Jose Bonifacio,2951.946452
Vila Jacui,2980.799235
Guaianases,3013.133915


In [11]:
join = modo_distri.join(distrito_renda)
join

,Bicicleta,Total,Bicicleta (%),RENDA
NOME_O,,,,
Cidade Ademar,67.0,410738.0,0.02,3418.158462
Capao Redondo,269.0,475712.0,0.06,3389.284922
Tucuruvi,272.0,208975.0,0.13,5938.093812
Jabaquara,284.0,504911.0,0.06,6002.553526
Vila Medeiros,356.0,167625.0,0.21,4346.540190
Vila Matilde,434.0,188149.0,0.23,3803.812061
Campo Grande,472.0,260019.0,0.18,6005.065945
Brasilandia,485.0,376310.0,0.13,3156.510088
Cidade Tiradentes,660.0,360709.0,0.18,2812.517945


In [10]:
join.corr()

,Bicicleta,Total,Bicicleta (%),RENDA
Bicicleta,1.000000,0.190898,0.790813,0.190419
Total,0.190898,1.000000,-0.290611,0.069306
Bicicleta (%),0.790813,-0.290611,1.000000,0.053798
RENDA,0.190419,0.069306,0.053798,1.000000
